# Metaclasses

Todas as classes são instâncias de `type`.

In [4]:
type('Olá, mundo!')

str

In [5]:
type(type('Olá, mundo!'))

type

In [6]:
type(type(type('Olá, mundo!')))

type

In [9]:
issubclass(type, object), issubclass(object, type)

(True, False)

In [10]:
isinstance(type, object), isinstance(object, type)

(True, True)